In [1]:
import json
import re
import numpy as np

In [2]:
def retrieve_data():
    
    en_prop_key = {
        'budget':'लागत',
        'narr locations' : 'कथा स्थल',
        'production company' : 'उत्पादन कंपनी',
        'artist' : 'कार्यकारी निर्माता',
        'ICAA rating' : 'आईसीएए रेटिंग',
        'series' : 'श्रृंखला का हिस्सा',
        'NMHH rating' : 'एनएमएचएच फिल्म रेटिंग' ,
        'nominated' : 'के लिए मनोनीत',
        'sequel' : 'बाद' ,
        'MPAA rating' : 'एमपीएए फिल्म रेटिंग',
        'date' : 'प्रकाशन तिथि',
        'release year' : 'प्रकाशन तिथि',
        'awards won' : 'पुरस्कार प्राप्त',
        'cast' : 'कास्ट मेंबर',
        'producer' : 'निर्माता' ,
        'website' : 'आधिकारिक वेबसाइट' ,
        'distributor' : 'द्वारा वितरित' ,
        'based on' : 'आधार पर',
        'director' : 'निदेशक',
        'country' : 'मूल देश',
        'editor' : 'फिल्म संपादक',
        'production designer' : 'प्रोडक्शन डिजाइनर',
        'singers' : 'संगीतकार',
        'topic' : 'मुख्य विषय',
        'fil locations' : 'फिल्मांकन स्थान',
        'music director' : 'कलाकार',
        'dir photo' : 'फोटोग्राफी के निदेशक',
        'FSK rating' : 'एफएसके फिल्म रेटिंग',
        'duration' : 'पहलू अनुपात',
        'colour': 'रंग',
        'box office' : 'बॉक्स ऑफिस',
        'costume designer' : 'कॉस्ट्यूम डिजाइनर',
        'lang' : 'फिल्म या टीवी शो की मूल भाषा',
        'screenwriter' : 'पटकथा लेखक' ,
        'genre' : 'शैली' 
    }

    
    
    sentence_templates = []
    with open('../Data/Films/Films_Generated_Wikipages/template_sents.json') as f:
        sentence_templates = json.load(f)

    hi_labelled_films = {}
    with open('../Data/Films/Films_Hindi_Wikidata_Hi_Labelled/films_hi_labelled.json') as f:
        hi_labelled_films = json.load(f)
    
    film_ids = []
    with open('../Data/Films/WikiBio_Styled_Formatted_Data/train/train.id') as f:
        film_ids = f.readlines()
    film_ids = [x.strip() for x in film_ids]
    
    film_titles = []
    with open('../Data/Films/WikiBio_Styled_Formatted_Data/train/train.title') as f:
        film_titles = f.readlines()
    film_titles = [x.strip() for x in film_titles]
    
    film_urls = []
    with open('../Data/Films/WikiBio_Styled_Formatted_Data/train/train.url') as f:
        film_urls = f.readlines()
    film_urls = [x.strip() for x in film_urls]
    
    film_details = {}
    
    for i in range(0,len(film_ids)):
        film_details[film_ids[i]] = {'url':film_urls[i], 'title': film_titles[i]}
        
    return film_details,sentence_templates, hi_labelled_films, en_prop_key

In [3]:
def get_first_sentences_templates(sentence_templates):
    first_sentences = []
    for sentence in sentence_templates:
        if '{{film}}' in sentence.split():
            first_sentences.append(sentence)
    
    return first_sentences

In [4]:
def retrieve_template_tags(sentence_templates):
    template_tags = {}
    for sentence_template in sentence_templates:
        template_tags[sentence_template] = re.findall(r'\{\{(.*?)\}\}', sentence_template)
    
    return template_tags

In [5]:
def retrieve_first_sentence_template(template_tags, film_wiki_props, en_prop_key):
    sentence_templates = list(template_tags.keys())
    first_sentences = get_first_sentences_templates(sentence_templates)
    first_sentences_tags = retrieve_template_tags(first_sentences)
    props = list(film_wiki_props.keys())
    
    for sent in first_sentences_tags.keys():
        tags = first_sentences_tags[sent]
        tags.remove('film')
        first_sentences_tags[sent] = tags
    
    matching_template = ""
    max_template_size = 0
    
    for first_sent in first_sentences_tags.keys():
        tags  = first_sentences_tags[first_sent]
        hi_tags = [en_prop_key[tag] for tag in tags]
        
        if set(hi_tags) <= set(props) and len(hi_tags) > max_template_size:
            max_template_size = len(hi_tags)
            matching_template = first_sent
            
    return matching_template
    

In [6]:
def create_wikipage(film_id, film_details, sentence_templates, hi_labelled_films, en_prop_key):
    template_tags = retrieve_template_tags(sentence_templates)
    wiki_props = np.array(list(en_prop_key.values()))
    wiki_props = list(np.unique(wiki_props))
    
    film = hi_labelled_films[film_id]
    film_wiki_props = {}
    
    for prop in wiki_props:
        if prop in film.keys():
            film_wiki_props[prop] = film[prop]
    
    article = retrieve_first_sentence_template(template_tags, film_wiki_props, en_prop_key)
    if article == "":
        article = "{{film}} एक फिल्म है।"
    
    return article

In [7]:
film_details,sentence_templates, hi_labelled_films, en_prop_key = retrieve_data()
first_sents = {}

In [8]:
create_wikipage('Q7500461', film_details, sentence_templates, hi_labelled_films, en_prop_key)

Q64768634
Q11170402
Q7500461
Q11183441
Q45273
Q7489422
Q50090551
Q11032364
Q5642734
Q16247933
Q5520836
Q592915
Q10956012
Q10970791
Q13564796
Q2698933
Q11007947
Q5104916
Q20058717
Q3521744
Q12438977
Q11000078
Q12416541
Q1553017
Q4734595
Q11038454
Q7044825
Q15983039
Q18146264
Q11014668
Q8050263
Q11112264
Q4900722
Q6736054
Q4444
Q1442306
Q12433149
Q27306757
Q11032567
Q62815320
Q6279006
Q10951424
Q11046952
Q6123574
Q4769893
Q852001
Q11047430
Q815550
Q6109605
Q3506295
Q6349887
Q10956161
Q3104859
Q8065778
Q7399477
Q5555802
Q5222008
Q4662841
Q174284
Q18547944
Q6036981
Q3419762
Q12938638
Q5018940
Q11162552
Q5104932
Q330113
Q4682452
Q10961083
Q936421
Q4791549
Q6308185
Q4900482
Q11205015
Q652186
Q4791575
Q190145
Q10969122
Q4514940
Q12432234
Q5584273
Q4842483
Q10976583
Q11112061
Q11108224
Q55080162
Q6720926
Q4673650
Q11015226
Q5308220
Q477299
Q776808
Q313884
Q58411779
Q60738336
Q11161484
Q6480009
Q5276609
Q11056433
Q18703155
Q16247587
Q12425234
Q11040114
Q11182884
Q5614606
Q12523677
Q3627595
Q109

Q12447578
Q232032
Q780886
Q10967181
Q4646285
Q924135
Q5506386
Q19824742
Q4662781
Q160060
Q22665878
Q7265844
Q5266623
Q13897247
Q7695179
Q7209457
Q11038914
Q608567
Q25302739
Q47516672
Q3135932
Q745973
Q10953530
Q21646922
Q5350172
Q6929026
Q47703
Q3429837
Q1549110
Q7189557
Q4690368
Q10953957
Q300809
Q18170549
Q2680157
Q15705611
Q11108656
Q42530057
Q7260792
Q7810145
Q7701713
Q531340
Q11021562
Q4662760
Q3157001
Q19605737
Q3423533
Q10967847
Q11016537
Q599420
Q3418011
Q3415825
Q4837199
Q3419976
Q5068476
Q27959206
Q5522558
Q582992
Q10952270
Q222867
Q10999193
Q11004095
Q5267264
Q4761903
Q797523
Q5350165
Q12435037
Q7395632
Q12423292
Q11010062
Q18354842
Q6469238
Q5575989
Q6819146
Q539269
Q1676775
Q25303001
Q42051
Q12428942
Q11006031
Q12454971
Q6050025
Q5350458
Q11026588
Q3764194
Q10953277
Q14786843
Q7180172
Q4661343
Q7423872
Q7399560
Q12454354
Q6817863
Q310204
Q300814
Q24807410
Q12449463
Q57982486
Q6981196
Q11263752
Q16826307
Q10973290
Q19829521
Q7286287
Q229633
Q12441542
Q20646924
Q7531719
Q642